In [42]:
import praw
import re
import numpy as np

In [9]:


# Initialize the Reddit client
reddit = praw.Reddit(
    client_id="h1oT9ECrvYgR8oipbCwXRg",
    client_secret="wq3QJIj0mcSB9TAIspKMG7pXTbhpLQ",
    user_agent="isreal_data"
)


In [10]:
def clean_text(text):
    # Remove URLs
    
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n\n.+?\n\n','', text, flags=re.MULTILINE)
    # Remove special characters (keeping only letters, numbers, and basic punctuation)
    text = re.sub(r'[^A-Za-z0-9\s,.\'"]+', ' ', text)
    text = re.sub(r'\n',' ', text)
    text = re.sub(r'n\'t',' not', text)
    text = re.sub(r'\'m', ' am', text)
    text = re.sub(r'\'s',' is', text)
    text = re.sub(r'[".,\']+', '', text)
    text = " ".join(text.split())
    text = text.lower()
    return text

In [16]:
# Specify the subreddit you want to pull data from
subreddit = reddit.subreddit("CryptoCurrency")
post_lst = []
# Retrieve the top 10 posts in the subreddit
for post in subreddit.top(time_filter="day", limit=100):  # Can use .new(), .top(), etc.
    text = post.title + " " + post.selftext
    text = clean_text(text)
    post_lst.append(text)
    print(text)

ancient bitcoin wallet comes alive after lying low for over 13 years moves 3670000 in btc at 646618 profit the daily hodl
gary gensler releases statement suggesting he is leaving the sec
luckiest trader ever from 16 to 3 million in 2 weeks on a meme coin decrypt
michael saylor doubts bitcoin will dip below 60k predicts 100k milestone by 2025
pennsylvania house introduces bill to implement a strategic bitcoin reserve
senator lummis proposal for bitcoin act sell fed s gold as btc overtakes silver
tax free crypto in us donald trump s plan to eliminate tax for xrp ada and more bullish
dot feels like it s been abandoned does anyone here think it s strange that one of the co founders of eth one of the best coins out there the guy who made dot made such a shitty coin it honestly made it even worse when i saw vitalik buterin s portfolio and he doesn t even have a single dot coin even meme coins see more action than dot
goldman sachs holds 461 million in blackrock is ibit new filing reveals
fed

In [12]:
from transformers import BertTokenizer, BertModel
import torch

C:\Users\bibek\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [18]:

# Extract embeddings in batches
test_embeddings = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Tokenize the batch of sentences at once
    for text in post_lst:
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs)
        # Get the [CLS] token embeddings for each sentence in the batch
        cls_embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
        test_embeddings.extend(cls_embeddings)

In [22]:
len(test_embeddings)

63

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SentimentClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [26]:
input_dim = 768  # From your embedding shape
hidden_dim = 128  # Adjust as needed
output_dim = 3  # Assuming 3 sentiment classes (-1, 0, 1)

In [27]:
model = SentimentClassifier(input_dim, hidden_dim, output_dim)  # Re-instantiate the model
model.load_state_dict(torch.load("sentiment_model.pth"))

<All keys matched successfully>

In [29]:
# Convert `val_embeddings` to a PyTorch tensor
X_test = torch.tensor(test_embeddings, dtype=torch.float32)

C:\Users\bibek\AppData\Local\Temp\ipykernel_10628\314498568.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  X_test = torch.tensor(test_embeddings, dtype=torch.float32)


In [30]:
# Make predictions
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)  # Get predicted class indices

In [34]:
predicted[7]

tensor(0)

In [35]:
post_lst[7]

'dot feels like it s been abandoned does anyone here think it s strange that one of the co founders of eth one of the best coins out there the guy who made dot made such a shitty coin it honestly made it even worse when i saw vitalik buterin s portfolio and he doesn t even have a single dot coin even meme coins see more action than dot'

RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Long

tensor([1, 1, 1, 1, 1, 1, 2, 0, 1, 0, 2, 1, 2, 0, 1, 2, 2, 0, 2, 0, 1, 1, 0, 1,
        1, 1, 2, 0, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2,
        1, 2, 2, 1, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2])